<a href="https://colab.research.google.com/github/Sazul19/soil_analysis_automation_system/blob/main/enhanced_model_prototype_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Load Pre-Trained EfficientNetB0 Model
def build_model(num_classes):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False  # Freeze the base model

    # Add Custom Layers for Color Classification
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(num_classes, activation='softmax')(x)  # Adjust num_classes for your color categories

    model = Model(inputs=base_model.input, outputs=predictions)
    return model


# Prepare the Data
def prepare_data(train_dir, val_dir, batch_size=32, img_size=(224, 224)):
    datagen = ImageDataGenerator(
        rescale=1.0 / 255.0,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2
    )

    train_generator = datagen.flow_from_directory(
        train_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )

    val_generator = datagen.flow_from_directory(
        val_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )

    return train_generator, val_generator


# Training Configuration
def train_model(model, train_generator, val_generator, epochs=20, batch_size=32, learning_rate=0.001):
    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint('efficientnet_color_model.h5', monitor='val_loss', save_best_only=True)

    # Train the model
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=epochs,
        steps_per_epoch=train_generator.samples // batch_size,
        validation_steps=val_generator.samples // batch_size,
        callbacks=[early_stopping, model_checkpoint]
    )

    return history


# Main Execution
if __name__ == "__main__":
    # Define paths
    train_dir = 'path/to/train/data'  # Replace with your train data path
    val_dir = 'path/to/validation/data'  # Replace with your validation data path

    # Prepare data
    train_generator, val_generator = prepare_data(train_dir, val_dir)

    # Build model
    num_classes = len(train_generator.class_indices)  # Dynamically calculate number of color categories
    model = build_model(num_classes)

    # Train the model
    history = train_model(model, train_generator, val_generator)

    # Unfreeze the base model for fine-tuning
    model.layers[0].trainable = True
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Fine-tune the model
    history_fine = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=10,
        steps_per_epoch=train_generator.samples // 32,
        validation_steps=val_generator.samples // 32,
        callbacks=[early_stopping, model_checkpoint]
    )
